# WeatherPy
----

In this analysis, we utilized Python code in conjunction with open access API systems to curate weather and geographic coordinates of 500+ cities located around the globe. By cleaning and analyzing data gathered from OpenWeather API, some conclusions could be made on the basis of city weather in relation to the location of the equator:

1. There were distinctive differences in the relationship of maximum ambient temperature and latitude coordinates between the northern and southern hemispheres. The northern hemisphere had a defined negative relationship as latitude increased, with the correlation coefficient equaling -1.42, meaning that as the latitude increased and cities were located further away from the equator, the temperatures dropped substantially. This would make sense as this data was collected in February 2021 and the northern hemisphere is in it's winter solstice during that time. The opposite can be observed with the analysis of temperature and latitude in the southern hemisphere, which had a slightly positive relationship with a positive correlation coefficient of 0.28. As the latitude is further away from the equator maximum temperature is lower, however there is a weaker coefficient for the southern hemisphere in February 2021 as it is currently within it's summer solstice so on average the temperatures are higher.

2. For both hemispheres, there seems to be minimal influence or correlations between percent humidity and a city's latitude coordinates. With very low correlation coefficients (less than 0.5) displayed in the linear regression analysis, observing relationships based on temperatures would be more accurate as humidity is dependent on the temperature levels.

3. Given that wind speed is affected by changes in air pressure in relation to temperature differences in the atmosphere, it makes sense that the southern hemisphere would have a slightly negative assocation with wind speed and latitude. As cities are located closer to the equator the air temperature is naturally higher, so there is less cool air dropping as warm air rises. The further away a city is from the equator the ambient temperature is generally cooler, so as warm air rises there is a greater change in air flow as cool air descends and increases overall wind speed.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy.stats as st
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

ModuleNotFoundError: No module named 'api_keys'

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

In [ ]:
#debugging only
# cities = cities[0:10]
# cities

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
#create query url to scan API
base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"


query_url = f"{base_url}&appid={weather_api_key}&units={units}&q="

In [ ]:
#list for analysis parameters
city_name = []
city_lat = []
city_lng = []
city_country =[]
city_date =[]
city_temp = []
city_humidity = []
city_cloud_cover = []
city_wind_speed = []

In [ ]:
print("Retrieving Desired City Data")
print(f"----------------------------------")
records = 0
records_set = 1

#for loop to go through each city from the API with try/except block so code doesn't break
for city in cities:
    city_url = f"{query_url}{city}"
    records = records + 1
    try:
        response = requests.get(city_url).json()
        time.sleep(0.5)
        print(f"Processing city number {records}")
        print(response)
        print()
        print()
        city_name.append(response["name"])
        city_lat.append(response["coord"]["lat"])
        city_lng.append(response["coord"]["lon"])
        city_country.append(response["sys"]["country"])
        city_date.append(response["dt"])
        city_temp.append(response["main"]["temp_max"])
        city_humidity.append(response["main"]["humidity"])
        city_cloud_cover.append(response["clouds"]["all"])
        city_wind_speed.append(response["wind"]["speed"])
        
        #Conditional for group city outputs 
        if records > 50:
            records_set += 1
            records = 1
    except:
        print(f"City not found")

print(f"----------------------------------")
print(f"End of Data Retrieval Process")
print(f"----------------------------------")

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
city_weather_df = pd.DataFrame({"City":city_name, "Latitude":city_lat, "Longitude":city_lng, "Country":city_country, "Date":city_date, "Max Temperature":city_temp, "Humidity":city_humidity, "Cloudiness":city_cloud_cover, "Wind Speed":city_wind_speed})
city_weather_df.head()

In [ ]:
city_weather_df.to_csv("City_Weather.csv", index=False, header=True)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
city_weather_df["Humidity"].max()

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
#Assign data to new variables
latitude = city_weather_df["Latitude"]
temperature = city_weather_df["Max Temperature"]

#Plot scatter plot with x and y values
plt.figure(figsize = (20,10))
plt.scatter(latitude, temperature)

#create x- and y-axis labels and a chart title
plt.title(f"City Latitude vs. Maximum Temperature (%s)" % time.strftime("%x"), fontsize = 20)
plt.xlabel("Latitude", fontsize = 15)
plt.ylabel("Maximum Temperature (F)", fontsize = 15)

plt.savefig("../Images/Latitude_vs_Max_Temp_Plot.png")
plt.show()

This code is visualizing the maximum temperatures (F) of the 600+ cities found in comparison to their latitude coordinates. This scatter plot is showing that as the latitude moves further away from 0 (the equator), the maximum temperature declines for both positive and negative latitudes.

## Latitude vs. Humidity Plot

In [ ]:
#Assign new variables
latitude = city_weather_df["Latitude"]
humidity = city_weather_df["Humidity"]

#Plot figure
plt.figure(figsize = (20, 10))
plt.scatter(latitude, humidity)

#chart labels and save plot image
plt.title(f"City Latitude vs. Percent Humidity (%s)" % time.strftime("%x"), fontsize = 20)
plt.xlabel("Latitude", fontsize=15)
plt.ylabel("Humidity (%)", fontsize = 15)

plt.savefig("../Images/Latitude_vs_Humidity_Plot.png")
plt.show()

This code is visualizing the percent humidity measurements of the 600+ random cities in comparison to their latitude coordinates. This scatter plot is indicating that humidity may not be dependent on latitude, as there are high and low humidity percentages for both city coordinates closer the the equator and those that are located further away. 

## Latitude vs. Cloudiness Plot

In [ ]:
#define variables
latitude = city_weather_df["Latitude"]
cloudiness = city_weather_df["Cloudiness"]

#plot figure
plt.figure(figsize = (20,10))
plt.scatter(latitude, cloudiness)

#designate labels and save as png file
plt.title(f"City Latitude vs. Percent Cloudiness(%s)" % time.strftime("%x"), fontsize = 20)
plt.xlabel("Latitude", fontsize = 15)
plt.ylabel("Cloudiness (%)", fontsize = 15)

plt.savefig("../Images/Latitude_vs_Cloudiness_Plot.png")
plt.show()

This code is visualizing the percent cloudiness of 600+ randomly selected cities around the world in relation to their latitude coordinates. This scatter plot is indicating that there may not be a strong assocation between percent cloud cover and latitude coordinates, as the points are widely distributed across the graph with high and low percentages at coordinates closer and further away from the equator.

## Latitude vs. Wind Speed Plot

In [ ]:
#define variables
latitude = city_weather_df["Latitude"]
wind_speed = city_weather_df["Wind Speed"]

#plot figure
plt.figure(figsize = (20,10))
plt.scatter(latitude, wind_speed)

#assign labels and save to png file
plt.title(f"City Latitude vs. Wind Speed (%s)" % time.strftime("%x"), fontsize = 20)
plt.xlabel("Latitude", fontsize = 15)
plt.ylabel("Wind Speed (MPH)", fontsize = 15)

plt.savefig("../Images/Latitude_vs_Wind_Speed.png")
plt.show()

This code is visualizing the relationship between the latitude coordinates of 600+ randomly selected cities around the globe and their calculated wind speeds, measured in mph. This scatter plot is indicating that generally speaking there are few outliers of high wind speeds across all latitude coordinates, meaning that wind speed may be attributed to different factors beyond latitude.

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
#Use .loc() function to filter for city latitudes above the equator
n_lats = city_weather_df.loc[(city_weather_df["Latitude"] > 0)]

#define variables
north_latitude = n_lats["Latitude"]
north_max_temp = n_lats["Max Temperature"]

#designate linear regression between latitude and max temp
temp_lat_slope, temp_lat_int, temp_lat_r, temp_lat_p, temp_lat_std_err = st.linregress(north_latitude, north_max_temp)

#create slope intercept equation
temp_lat_best_fit = temp_lat_slope * north_latitude + temp_lat_int

#convert to y=mx+b format for graph
north_temp_equation = "y=" + str(round(temp_lat_slope, 2)) + "x+" + str(round(temp_lat_int, 2))

#plot figure
plt.figure(figsize = (20,10))
plt.scatter(north_latitude, north_max_temp)

#Plot linear regression
plt.plot(north_latitude, temp_lat_best_fit, "--", color = "red")

#plot y=mx+b equation on chart
plt.annotate(north_temp_equation, (0, 50), fontsize = 15, color="red")

#assign labels and save to png file
plt.title(f"Northern Hemisphere Max Temperature vs Latitude (%s)" % time.strftime("%x"), fontsize = 20)
plt.xlabel("Latitude", fontsize = 15)
plt.ylabel("Maximum Temperatures (F)", fontsize = 15)

#include r-value in output
print(f"r=value: {temp_lat_r}")

plt.savefig("../Images/North_Hem_Max_Temp_vs_Lat_Plot.png")
plt.show()


This code is visualizing the relationship between maximum temperature of 600+ random cities around the globe and their associated latitude coordinates in the northern hemisphere. This scatter plot indicates that as cities are located further away from the equator, their maximum temperature decreases which is expected.

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
#.loc() for cities below equator and define variables
s_lats = city_weather_df.loc[city_weather_df["Latitude"] <0]

south_latitude = s_lats["Latitude"]
south_max_temp = s_lats["Max Temperature"]

#linear regression/slope-intercept 
s_lat_slope, s_lat_int, s_lat_r, s_lat_p, s_lat_std_err = st.linregress(south_latitude, south_max_temp)

s_lat_fit = s_lat_slope * south_latitude + s_lat_int

#y=mx+b equation
s_lat_equation = "y=" + str(round(s_lat_slope, 2)) + "x+" + str(round(s_lat_int, 2))

#plot figure
plt.figure(figsize = (20,10))
plt.scatter(south_latitude, south_max_temp)

#Plot linear regression
plt.plot(south_latitude, s_lat_fit, "--", color="red")

#Add y=mx+b to chart
plt.annotate(s_lat_equation, (-45,80), color="red", fontsize = 15)

#Assign labels and save as png file
plt.title(f"Southern Hemisphere Max Temperatures vs Latitude (%s)" % time.strftime("%x"), fontsize = 20, )
plt.xlabel("Latitude", fontsize = 15)
plt.ylabel("Maximum Temperatures (F)", fontsize = 15)

#designate r-value
print(f"r-value: {s_lat_r}")
      
plt.savefig("../Images/South_Hem_Max_Temp_vs_Lat_Plot.png")
plt.show()

This code is visualizing the relationship between maximum temperature of 600+ random cities around the globe and their associated latitude coordinates in the southern hemisphere. This scatter plot indicates that as cities move closer to the equator, there is a positive relationship between temperature and latitude, i.e. they experience higher temperatures which is expected.

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#Define variables
north_latitude = n_lats["Latitude"]
north_humidity = n_lats["Humidity"]

#linear regression/slope intercept
n_lat_slope, n_lat_int, n_lat_r, n_lat_p, n_lat_std_err = st.linregress(north_latitude, north_humidity)

n_lat_fit = n_lat_slope * north_latitude + n_lat_int

#y=mx+b equation
n_lat_equation = "y=" + str(round(n_lat_slope, 2)) + "x+" + str(round(n_lat_int, 2))

#plot figure
plt.figure(figsize = (15,10))
plt.scatter(north_latitude, north_humidity)

#Plot linear regression
plt.plot(north_latitude, n_lat_fit, "--", color="red")

#Add y=mx+b to chart
plt.annotate(n_lat_equation, (0,50), color="red", fontsize = 15)

#Assign labels and save as png file
plt.title(f"Northern Hemisphere Humidity vs Latitude (%s)" % time.strftime("%x"), fontsize = 20)
plt.xlabel("Latitude", fontsize = 15)
plt.ylabel("Humidity (%)", fontsize = 15)

#designate r-value
print(f"r-value: {n_lat_r}")
      
plt.savefig("../Images/North_Hem_Humidity_vs_Lat_Plot.png")
plt.show()


This code is visualizing the relationship between percent humidity of 600+ random cities around the globe and their associated latitude coordinates in the northern hemisphere. This scatter plot indicates that there is a minimal positive relationship between humidity levels and latitude coordinates, as some locations closer to the equator have lower humidity percentage and some locations further from the equator experience higher humidity percentages. Given that there is a wide array of city points beyond the best fit line of this linear regression, there may not be any definitive conclusions to be made on the association of latitude and humidity.

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#Define variables
south_latitude = s_lats["Latitude"]
south_humidity = s_lats["Humidity"]

#linear regression/slope-intercept 
s_lat_slope, s_lat_int, s_lat_r, s_lat_p, s_lat_std_err = st.linregress(south_latitude, south_humidity)

s_lat_fit = s_lat_slope * south_latitude + s_lat_int

#y=mx+b equation
s_lat_equation = "y=" + str(round(s_lat_slope, 2)) + "x+" + str(round(s_lat_int, 2))

#plot figure
plt.figure(figsize = (20,10))
plt.scatter(south_latitude, south_humidity)

#Plot linear regression
plt.plot(south_latitude, s_lat_fit, "--", color="red")

#Add y=mx+b to chart
plt.annotate(s_lat_equation, (-45,80), color="red", fontsize = 15)

#Assign labels and save as png file
plt.title(f"Southern Hemisphere Humidity vs Latitude (%s)" % time.strftime("%x"), fontsize = 20, )
plt.xlabel("Latitude", fontsize = 15)
plt.ylabel("Humidity (%)", fontsize = 15)

#designate r-value
print(f"r-value: {s_lat_r}")
      
plt.savefig("../Images/South_Hem_Humidity_vs_Lat_Plot.png")
plt.show()


This code is visualizing the relationship between percent humidity of 600+ random cities around the globe and their associated latitude coordinates in the southern hemisphere. This scatter plot indicates that there is a minimal positive relationship between humidity levels and latitude coordinates, as some locations closer to the equator have lower humidity percentage and some locations further from the equator experience higher humidity percentages. Given that there is a wide array of city points beyond the best fit line of this linear regression, there may not be any definitive conclusions to be made on the association of latitude and humidity.

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#Define variables
north_latitude = n_lats["Latitude"]
north_cloudiness = n_lats["Cloudiness"]

#linear regression/slope intercept
n_lat_slope, n_lat_int, n_lat_r, n_lat_p, n_lat_std_err = st.linregress(north_latitude, north_cloudiness)

n_lat_fit = n_lat_slope * north_latitude + n_lat_int

#y=mx+b equation
n_lat_equation = "y=" + str(round(n_lat_slope, 2)) + "x+" + str(round(n_lat_int, 2))

#plot figure
plt.figure(figsize = (15,10))
plt.scatter(north_latitude, north_cloudiness)

#Plot linear regression
plt.plot(north_latitude, n_lat_fit, "--", color="red")

#Add y=mx+b to chart
plt.annotate(n_lat_equation, (35,60), color="red", fontsize = 15)

#Assign labels and save as png file
plt.title(f"Northern Hemisphere Cloudiness vs Latitude (%s)" % time.strftime("%x"), fontsize = 20)
plt.xlabel("Latitude", fontsize = 15)
plt.ylabel("Cloudiness (%)", fontsize = 15)

#designate r-value
print(f"r-value: {n_lat_r}")
      
plt.savefig("../Images/North_Hem_Cloud_vs_Lat_Plot.png")
plt.show()


This code is visualizing the relationship between percent cloudiness of 600+ random cities around the globe and their associated latitude coordinates in the northern hemisphere. This scatter plot indicates that there is a minimal positive relationship between humidity levels and latitude coordinates, as some locations closer to the equator have lower cloudiness percentage and some locations further from the equator experience higher cloudiness percentages. Given that there is a wide array of city points beyond the best fit line of this linear regression, there may not be any definitive conclusions to be made on the association of latitude and cloudiness.

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#Define variables
south_latitude = s_lats["Latitude"]
south_cloudiness = s_lats["Cloudiness"]

#linear regression/slope-intercept 
s_lat_slope, s_lat_int, s_lat_r, s_lat_p, s_lat_std_err = st.linregress(south_latitude, south_cloudiness)

s_lat_fit = s_lat_slope * south_latitude + s_lat_int

#y=mx+b equation
s_lat_equation = "y=" + str(round(s_lat_slope, 2)) + "x+" + str(round(s_lat_int, 2))

#plot figure
plt.figure(figsize = (20,10))
plt.scatter(south_latitude, south_cloudiness)

#Plot linear regression
plt.plot(south_latitude, s_lat_fit, "--", color="red")

#Add y=mx+b to chart
plt.annotate(s_lat_equation, (-42,55), color="red", fontsize = 15)

#Assign labels and save as png file
plt.title(f"Southern Hemisphere Cloudiness vs Latitude (%s)" % time.strftime("%x"), fontsize = 20, )
plt.xlabel("Latitude", fontsize = 15)
plt.ylabel("Cloudiness (%)", fontsize = 15)

#designate r-value
print(f"r-value: {s_lat_r}")
      
plt.savefig("../Images/South_Hem_Cloud_vs_Lat_Plot.png")
plt.show()

This code is visualizing the relationship between percent cloudiness of 600+ random cities around the globe and their associated latitude coordinates in the southern hemisphere. This scatter plot indicates that there is a minimal positive relationship between humidity levels and latitude coordinates, as some locations closer to the equator have lower cloudiness percentage and some locations further from the equator experience higher cloudiness percentages. Given that there is a wide array of city points beyond the best fit line of this linear regression, there may not be any definitive conclusions to be made on the association of latitude and cloudiness.

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#Define variables
north_latitude = n_lats["Latitude"]
north_wind_speed = n_lats["Wind Speed"]

#linear regression/slope intercept
n_lat_slope, n_lat_int, n_lat_r, n_lat_p, n_lat_std_err = st.linregress(north_latitude, north_wind_speed)

n_lat_fit = n_lat_slope * north_latitude + n_lat_int

#y=mx+b equation
n_lat_equation = "y=" + str(round(n_lat_slope, 2)) + "x+" + str(round(n_lat_int, 2))

#plot figure
plt.figure(figsize = (15,10))
plt.scatter(north_latitude, north_wind_speed)

#Plot linear regression
plt.plot(north_latitude, n_lat_fit, "--", color="red")

#Add y=mx+b to chart
plt.annotate(n_lat_equation, (35,25), color="red", fontsize = 15)

#Assign labels and save as png file
plt.title(f"Northern Hemisphere Wind Speed vs Latitude (%s)" % time.strftime("%x"), fontsize = 20)
plt.xlabel("Latitude", fontsize = 15)
plt.ylabel("Wind Speed (mph)", fontsize = 15)

#designate r-value
print(f"r-value: {n_lat_r}")
      
plt.savefig("../Images/North_Hem_Wind_vs_Lat_Plot.png")
plt.show()


This code is visualizing the relationship between wind speed (mph) of 600+ random cities around the globe and their associated latitude coordinates in the northern hemisphere. This scatter plot indicates that there is a minimal positive relationship between humidity levels and latitude coordinates, as some locations closer to the equator have lower wind speeds and some locations further from the equator experience higher wind speeds. Given that the slope of the best fit line is as low as 0.04, it is indicative of a weaker association between latitude coordinates and average wind speeds.

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#Define variables
south_latitude = s_lats["Latitude"]
south_wind_speed = s_lats["Wind Speed"]

#linear regression/slope-intercept 
s_lat_slope, s_lat_int, s_lat_r, s_lat_p, s_lat_std_err = st.linregress(south_latitude, south_wind_speed)

s_lat_fit = s_lat_slope * south_latitude + s_lat_int

#y=mx+b equation
s_lat_equation = "y=" + str(round(s_lat_slope, 2)) + "x+" + str(round(s_lat_int, 2))

#plot figure
plt.figure(figsize = (20,10))
plt.scatter(south_latitude, south_wind_speed)

#Plot linear regression
plt.plot(south_latitude, s_lat_fit, "--", color="red")

#Add y=mx+b to chart
plt.annotate(s_lat_equation, (-50,12), color="red", fontsize = 15)

#Assign labels and save as png file
plt.title(f"Southern Hemisphere Wind Speed vs Latitude (%s)" % time.strftime("%x"), fontsize = 20, )
plt.xlabel("Latitude", fontsize = 15)
plt.ylabel("Wind Speed (mph)", fontsize = 15)

#designate r-value
print(f"r-value: {s_lat_r}")
      
plt.savefig("../Images/South_Wind_vs_Lat_Plot.png")
plt.show()

This code is visualizing the relationship between wind speeds of 600+ random cities around the globe and their associated latitude coordinates in the northern hemisphere. This scatter plot indicates that there is a minimal negative relationship between average wind speeds and latitude coordinates, as some locations closer to the equator have lower wind speeds and some locations further from the equator experience higher wind speeds. Given that there is a wide array of city points beyond the best fit line of this linear regression, and that the slope is as minimal as 0.06, there may not be any definitive conclusions to be made on the association of latitude and average wind speeds.